In [ ]:
import os

if not os.getcwd().endswith("src"):
    %cd ..
%pwd

/home/xqz-u/FACT/src


'/home/xqz-u/FACT/src'

In [ ]:
import itertools as it
from copy import deepcopy
import logging
import os
import pprint
from typing import Dict, Sequence

import implicit
from implicit import evaluation
import numpy as np
import pandas as pd
import scipy
from scipy import sparse

import config
import constants
import utils

logger = logging.getLogger()
utils.setup_root_logging()

logger.info(f"Implcit using GPU: {implicit.gpu.HAS_CUDA}")

p1660 [23-01-18 13:36] [INFO]  Implcit using GPU: True  (root:22)


In [ ]:
# from the README file
column_names = ["user", "item", "rating", "timestamp"]
user_item_df = pd.read_csv(config.MOVIELENS_DIR / "ratings.dat", names=column_names, sep="::", engine="python")
# user_item_df = user_item_df.rename(columns={"userID": "user", "artistID": "item"})
# user_item_df = user_item_df.pivot(index="user", columns="item", values="weight").fillna(0)
user_item_df

,user,item,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
# get rid of the timestamp column
user_item_df = user_item_df.drop(columns="timestamp")
# spread into user X item rating matrix
user_item_df = user_item_df.pivot(index="user", columns="item", values="rating").fillna(0)
user_item_df

item,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
k_users, k_items = 2000, 2500
# ratings are on a 1-5 scale, sum non-zero entries to get user/items with most ratings
bool_df = user_item_df.astype(bool)
# filter top 2000 most rated users
topk_users = bool_df.sum(axis=1).sort_values(ascending=False).index[:k_users]
user_item_df = user_item_df.loc[user_item_df.index.intersection(topk_users)]
# filter top 2500 most rated items 
topk_items = bool_df.sum(axis=0).sort_values(ascending=False).index[:k_items]
user_item_df = user_item_df[user_item_df.columns.intersection(topk_items)]
user_item_df

item,1,2,3,4,5,6,7,8,9,10,...,3932,3936,3937,3943,3946,3947,3948,3949,3950,3952
user,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,5.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025,5.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6035,4.0,0.0,1.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# remap ratings {3, 4, 5} to a range of 5, and set ratings < 3 to 0
ratings_remap = dict(zip(range(1, 6), np.linspace(3, 5, 5)))
user_item_df = user_item_df.applymap(lambda rating: ratings_remap.get(rating, 0))
user_item_df

item,1,2,3,4,5,6,7,8,9,10,...,3932,3936,3937,3943,3946,3947,3948,3949,3950,3952
user,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,5.0,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,4.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025,5.0,0.0,4.0,4.0,0.0,0.0,4.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6035,4.5,0.0,3.0,3.5,3.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,3.5,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
